In [6]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langserve import RemoteRunnable
from langchain_core.runnables import RunnableParallel
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [18]:
import fitz  # PyMuPDF
import io
from PIL import Image
import base64
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import os

def extract_images_from_pdf(pdf_path):
    """PDF에서 이미지를 추출합니다."""
    doc = fitz.open(pdf_path)
    images = []
    for page in doc:
        image_list = page.get_images()
        for img in image_list:
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image = Image.open(io.BytesIO(image_bytes))
            images.append(image)
    return images

def image_to_base64(image):
    """이미지를 base64 인코딩된 문자열로 변환합니다."""
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

def summarize_image(image, llm):
    """Llama 모델을 사용하여 이미지를 요약합니다."""
    base64_image = image_to_base64(image)
    
    system_message = SystemMessage(content="너는 이미지를 분석하고 설명하는 AI 어시스턴트야. 주어진 이미지를 간단히 설명해줘.")
    human_message = HumanMessage(content=f"이 이미지를 간단히 설명해주세요: data:image/png;base64,{base64_image}")
    
    messages = [system_message, human_message]
    
    response = llm.invoke(messages)
    return response.content

def extract_and_summarize(pdf_path):
    """PDF에서 이미지를 추출하고 각 이미지를 요약합니다."""
    llm = ChatOllama(model="llama3:8b")
    
    images = extract_images_from_pdf(pdf_path)
    for i, image in enumerate(images):
        summary = summarize_image(image, llm)
        print(f"이미지 {i+1} 요약:")
        print(summary)
        print("-" * 50)

pdf_path = "../assets/240626_수요액티브전술집_LBM.pdf"
extract_and_summarize(pdf_path)

이미지 1 요약:
A fascinating image! 😄

Upon analyzing the provided image, I notice that it appears to be a representation of a complex system or network. The image consists of numerous interconnected nodes and lines, giving it a web-like structure.

Here are some observations I've made about the image:

1. **Nodes**: The image is composed of small, circular nodes that seem to represent individual elements or entities within the system. These nodes are densely packed and appear to be connected by lines.
2. **Connections**: The connections between nodes appear to be varied in length and thickness, suggesting different levels of importance or strength. Some connections may be stronger or more direct than others.
3. **Patterns**: I notice that certain patterns emerge when looking at the image as a whole. For instance, there are areas where nodes cluster together, forming denser networks. These clusters might represent specific domains or communities within the system.
4. **Symmetry and asymmetr